<a href="https://colab.research.google.com/github/JaiswalFelipe/Dissertation-Project/blob/main/mydataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [5]:
!pip install imagecodecs 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import os
import sys
import numpy as np
import imageio
import torch
import torchvision
import math
import scipy.stats as stats
import torch.nn.functional as F
import cv2

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch.utils import data
from skimage import transform
from skimage import img_as_float
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torchvision import transforms
from skimage import transform
from torch import nn
from torchvision import models

#from data_utils import create_or_load_statistics, create_distrib, normalize_images, data_augmentation

scaler = MinMaxScaler()

# Model

# data_utils

In [9]:
def compute_image_mean(data):
    _mean = np.mean(np.mean(np.mean(data, axis=0), axis=0), axis=0)
    _std = np.std(np.std(np.std(data, axis=0, ddof=1), axis=0, ddof=1), axis=0, ddof=1)

    return _mean, _std
    

def normalize_images(data, _mean, _std):
    for i in range(len(_mean)):
        data[:, :, i] = np.subtract(data[:, :, i], _mean[i])
        data[:, :, i] = np.divide(data[:, :, i], _std[i])



def data_augmentation(img, label=None):
    rand_fliplr = np.random.random() > 0.50
    rand_flipud = np.random.random() > 0.50
    rand_rotate = np.random.random()

    if rand_fliplr:
        img = np.fliplr(img)
        label = np.fliplr(label)
    if rand_flipud:
        img = np.flipud(img)
        label = np.flipud(label)

    if rand_rotate < 0.25:
        img = transform.rotate(img, 270, order=1, preserve_range=True)
        label = transform.rotate(label, 270, order=0, preserve_range=True)
    elif rand_rotate < 0.50:
        img = transform.rotate(img, 180, order=1, preserve_range=True)
        label = transform.rotate(label, 180, order=0, preserve_range=True)
    elif rand_rotate < 0.75:
        img = transform.rotate(img, 90, order=1, preserve_range=True)
        label = transform.rotate(label, 90, order=0, preserve_range=True)

    img = img.astype(np.float32)
    label = label.astype(np.int64)

    return img, label


# utils

# Dataset

### training_data

- should the vali data be reconsstructed? since we are only getting from training foler itself?


In [7]:
class NGDataset(data.Dataset):
  def __init__(self, dataset_input_path, images, output_path):

    self.dataset_input_path = dataset_input_path
    self.images = images
    #self.img_dir = img_dir
    #self.mask_dir = mask_dir
    #self.images = os.listdir(img_dir)
    #self.masks = os.listdir(mask_dir)

    self.output_path = output_path


    print("debug1")
    # data and label
    self.data, self.labels = self.load_images()
    print("debug2")
    #print(self.data.ndim, self.data.shape, self.data[0].shape, np.min(self.data), np.max(self.data),
    #      self.labels.shape, np.bincount(self.labels.astype(int).flatten()))

    if self.data.ndim == 4:  # if all images have the same shape
       self.num_channels = self.data.shape[-1]  # get the number of channels
    else:
       self.num_channels = self.data[0].shape[-1]  # get the number of channels

    self.num_classes = 2  # binary - two classes
    # negative classes will be converted into 2 so they can be ignored in the loss
    self.labels[np.where(self.labels < 0)] = 2
    
    print('num_channels and labels', self.num_channels, self.num_classes, np.bincount(self.labels.flatten()))

    #self.distrib, self.gen_classes = self.make_dataset()

    self.mean, self.std = compute_image_mean(self.data)

  print("debug3")

  def load_images(self):
        images = []
        masks = []
        for img in self.images:
            temp_image = imageio.imread(os.path.join(self.dataset_input_path, img + '')).astype(np.float64)
            temp_image[np.where(temp_image < -1.0e+38)] = 0 # remove extreme negative values (probably NO_DATA values)
            
            temp_mask = imageio.imread(os.path.join(self.dataset_input_path, img + '')).astype(int)
            temp_mask[np.where(temp_mask < -1.0e+38)] = 0

            images.append(temp_image)
            masks.append(temp_mask)

        return np.asarray(images), np.asarray(masks)

  #def load_images(self):
  #  images = []
  #  masks = []
  #  for img in os.listdir(self.img_dir):
  #    image = imageio.imread(os.path.join(self.img_dir, img)).astype(np.float64)
   #   image[np.where(image < -1.0e+38)] = 0 # remove extreme negative values (probably NO_DATA values)
   #   
   #   images.append(image)

  #  for msk in os.listdir(self.mask_dir):
  #   m = imageio.imread(os.path.join(masks, msk)).astype(int)
  #    masks.append(m)

  #  return np.asarray(images), np.asarray(masks) 

  print("debug4")

  def __getitem__(self, index):
    
    #Reading items from list.
    img = self.data[index]
    label = self.labels[index]

    # Normalization.
    normalize_images(img, self.mean, self.std) # check data_utils.py
    
    # Data augmentation
    img, label = data_augmentation(img, label)
     
    img = np.transpose(img, (2, 0, 1))

    # Turning to tensors.
    img = torch.from_numpy(img.copy())
    label = torch.from_numpy(label.copy())

    # Returning to iterator.
    return img.float(), label

  def __len__(self):
    return len(self.data)

  print("debug5")


debug3
debug4
debug5


In [10]:
input_path = '/content/drive/MyDrive/test2/img'
#mask_path = '/content/drive/MyDrive/test2/msk'
output_path = '/content/drive/MyDrive/Dissertation/outputs/'

images = os.listdir(input_path)

train_loader = NGDataset(input_path, images, output_path)


debug1
debug2
num_channels and labels 23 2 [1847769  132706  631393 ...       0       0       1]


# training

In [ ]:
import albumentations as A
import torch.optim as optim

from albumentations.pytorch import ToTensorsV2
from tqdm import tqdm 

#from utils import (
#    load_checkpoint,
#    save_checkpoint,
#    get_loaders,
#    check_accuracy
#    save_predictions_as_imgs
#)


def train(train_loader, model, criterion, optimizer, epoch):
    # Setting network for training mode.
    model.train()

    # Average Meter for batch loss.
    train_loss = list()

    # Iterating over batches.
    for i, data in enumerate(train_loader):
        # Obtaining images, labels and paths for batch.
        inps, labels = data[0], data[1]

        # if the current batch does not have samples from all classes
        # print('out i', i, len(np.unique(labels.flatten())))
        # if len(np.unique(labels.flatten())) < 10:
        #     print('in i', i, len(np.unique(labels.flatten())))
        #     continue

        # Casting tensors to cuda.
        inps = Variable(inps).cuda()
        labs = Variable(labels).cuda()

        # Clears the gradients of optimizer.
        optimizer.zero_grad()

        # Forwarding.
        outs = model(inps)

        # Computing loss.
        loss = criterion(outs, labs)

        if math.isnan(loss):
            print('-------------------------NaN-----------------------------------------------')
            print(inps.shape, labels.shape, outs.shape, np.bincount(labels.flatten()))
            print(np.min(inps.cpu().data.numpy()), np.max(inps.cpu().data.numpy()),
                  np.isnan(inps.cpu().data.numpy()).any())
            print(np.min(labels.cpu().data.numpy()), np.max(labels.cpu().data.numpy()),
                  np.isnan(labels.cpu().data.numpy()).any())
            print(np.min(outs.cpu().data.numpy()), np.max(outs.cpu().data.numpy()),
                  np.isnan(outs.cpu().data.numpy()).any())
            print('-------------------------NaN-----------------------------------------------')
            raise AssertionError

        # Computing backpropagation.
        loss.backward()
        optimizer.step()

        # Updating loss meter.
        train_loss.append(loss.data.item())

        # Printing.
        if (i + 1) % DISPLAY_STEP == 0:
            soft_outs = F.softmax(outs, dim=1)
            # Obtaining predictions.
            prds = soft_outs.cpu().data.numpy().argmax(axis=1).flatten()

            labels = labels.cpu().data.numpy().flatten()

            # filtering out pixels
            coord = np.where(labels != train_loader.dataset.num_classes)
            labels = labels[coord]
            prds = prds[coord]

            acc = accuracy_score(labels, prds)
            conf_m = confusion_matrix(labels, prds, labels=[0, 1])
            f1_s = f1_score(labels, prds, average='weighted')

            _sum = 0.0
            for k in range(len(conf_m)):
                _sum += (conf_m[k][k] / float(np.sum(conf_m[k])) if np.sum(conf_m[k]) != 0 else 0)

            print("Training -- Epoch " + str(epoch) + " -- Iter " + str(i + 1) + "/" + str(len(train_loader)) +
                  " -- Time " + str(datetime.datetime.now().time()) +
                  " -- Training Minibatch: Loss= " + "{:.6f}".format(train_loss[-1]) +
                  " Overall Accuracy= " + "{:.4f}".format(acc) +
                  " Normalized Accuracy= " + "{:.4f}".format(_sum / float(train_loader.dataset.num_classes)) +
                  " F1 Score= " + "{:.4f}".format(f1_s) +
                  " Confusion Matrix= " + np.array_str(conf_m).replace("\n", "")
                  )
            sys.stdout.flush()

    return sum(train_loss) / len(train_loss), _sum / float(train_loader.dataset.num_classes)


def main():
    parser = argparse.ArgumentParser(description='main')
    # general options
    parser.add_argument('--operation', type=str, required=True, help='Operation [Options: Train | Test]')
    parser.add_argument('--output_path', type=str, required=True,
                        help='Path to to save outcomes (such as images and trained models) of the algorithm.')

    # dataset options
    parser.add_argument('--dataset_path', type=str, required=True, help='Dataset path.')
    parser.add_argument('--training_images', type=str, nargs="+", required=True, help='Training image names.')
    parser.add_argument('--testing_images', type=str, nargs="+", required=True, help='Testing image names.')
    parser.add_argument('--crop_size', type=int, required=True, help='Crop size.')
    parser.add_argument('--stride_crop', type=int, required=True, help='Stride size')

    # model options
    parser.add_argument('--model_name', type=str, required=True,
                        choices=['deeplab', 'fcnwideresnet'], help='Model to evaluate')
    parser.add_argument('--model_path', type=str, default=None, help='Model path.')
    parser.add_argument('--learning_rate', type=float, default=0.01, help='Learning rate')
    parser.add_argument('--weight_decay', type=float, default=0.005, help='Weight decay')
    parser.add_argument('--batch_size', type=int, default=128, help='Batch size')
    parser.add_argument('--epoch_num', type=int, default=500, help='Number of epochs')

    # handling imbalanced data
    parser.add_argument('--loss_weight', type=float, nargs='+', default=[1.0, 1.0], help='Weight Loss.')
    parser.add_argument('--weight_sampler', type=str2bool, default=False, help='Use weight sampler for loader?')
    args = parser.parse_args()
    print(args)

    # Making sure output directory is created.
    pathlib.Path(args.output_path).mkdir(parents=True, exist_ok=True)

    # writer for the tensorboard
    writer = SummaryWriter(os.path.join(args.output_path, 'logs'))

    if args.operation == 'Train':
        print('---- training data ----')
        train_set = DataLoader('Train', args.dataset_path, args.training_images, args.crop_size, args.stride_crop,
                               args.output_path)
        print('---- testing data ----')
        test_set = DataLoader('Test', args.dataset_path, args.testing_images, args.crop_size, args.stride_crop,
                              args.output_path)

        if args.weight_sampler is False:
            train_loader = torch.utils.data.DataLoader(train_set, batch_size=args.batch_size,
                                                       shuffle=True, num_workers=NUM_WORKERS, drop_last=False)
        else:
            class_loader_weights = 1. / np.bincount(train_set.gen_classes)
            samples_weights = class_loader_weights[train_set.gen_classes]
            sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weights, len(samples_weights),
                                                                     replacement=True)
            train_loader = torch.utils.data.DataLoader(train_set, batch_size=args.batch_size,
                                                       num_workers=NUM_WORKERS, drop_last=False, sampler=sampler)

        test_loader = torch.utils.data.DataLoader(test_set, batch_size=args.batch_size,
                                                  shuffle=False, num_workers=NUM_WORKERS, drop_last=False)

        # Setting network architecture.
        model = model_factory(args.model_name, train_set.num_channels, train_set.num_classes).cuda()

        criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor(args.loss_weight),
                                        ignore_index=train_set.num_classes).cuda()

        # Setting optimizer.
        optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay,
                               betas=(0.9, 0.99))
        # optimizer = optim.SGD(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay, momentum=0.9)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

        curr_epoch = 1
        best_records = []
        if args.model_path is not None:
            print('Loading model ' + args.model_path)
            best_records = np.load(os.path.join(args.output_path, 'best_records.npy'), allow_pickle=True)
            model.load_state_dict(torch.load(args.model_path))
            # optimizer.load_state_dict(torch.load(args.model_path.replace("model", "opt")))
            curr_epoch += int(os.path.basename(args.model_path)[:-4].split('_')[-1])
            for i in range(curr_epoch):
                scheduler.step()
        model.cuda()

        # Iterating over epochs.
        print('---- training ----')
        for epoch in range(curr_epoch, args.epoch_num + 1):
            # Training function.
            t_loss, t_nacc = train(train_loader, model, criterion, optimizer, epoch)
            writer.add_scalar('Train/loss', t_loss, epoch)
            writer.add_scalar('Train/acc', t_nacc, epoch)
            if epoch % VAL_INTERVAL == 0:
                # Computing test.
                acc, nacc, f1_s, kappa, track_cm = test_full_map(test_loader, model, epoch, args.output_path)
                writer.add_scalar('Test/acc', nacc, epoch)
                save_best_models(model, args.output_path, best_records, epoch, kappa)
                # patch_acc_loss=None, patch_occur=None, patch_chosen_values=None
            scheduler.step()
    elif args.operation == 'Test':
        print('---- testing data ----')
        test_set = DataLoader('Test', args.dataset_path, args.training_images, args.crop_size, args.stride_crop,
                              args.output_path)
        test_loader = torch.utils.data.DataLoader(test_set, batch_size=args.batch_size,
                                                  shuffle=False, num_workers=NUM_WORKERS, drop_last=False)

        # Setting network architecture.
        model = model_factory(args.model_name, test_set.num_channels, test_set.num_classes).cuda()

        best_records = np.load(os.path.join(args.output_path, 'best_records.npy'), allow_pickle=True)
        index = 0
        for i in range(len(best_records)):
            if best_records[index]['kappa'] < best_records[i]['kappa']:
                index = i
        epoch = int(best_records[index]['epoch'])
        print("loading model_" + str(epoch) + '.pth')
        model.load_state_dict(torch.load(os.path.join(args.output_path, 'model_' + str(epoch) + '.pth')))
        model.cuda()

        test_full_map(test_loader, model, epoch, args.output_path)
    else:
        raise NotImplementedError("Process " + args.operation + "not found!")


if __name__ == "__main__":
    main()
